In [2]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('data/bangkok_traffy.csv')

# Randomly sample 1000 rows
sampled_df = df.sample(n=1000, random_state=42)

# Save to new CSV file
sampled_df.to_csv('1000samp.csv', index=False)


In [ ]:
import pandas as pd
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk
import requests
from io import BytesIO

# Load sampled data
df = pd.read_csv("1000samp.csv")
if 'label' not in df.columns:
    df['label'] = ""  # initialize only if not exists

current_index = 0

# Tkinter app setup
root = tk.Tk()
root.title("Labeling Tool")

img_label = tk.Label(root)
img_label.pack()

text_label = tk.Label(root, text="", wraplength=800, justify="left")
text_label.pack(pady=10)

def load_data(index):
    row = df.iloc[index]
    comment = row['comment']
    photo_url = row['photo']

    text_label.config(text=f"{index+1}/{len(df)}: {comment}")

    try:
        response = requests.get(photo_url)
        img = Image.open(BytesIO(response.content)).resize((500, 300))
        img = ImageTk.PhotoImage(img)
        img_label.img = img  # keep a reference!
        img_label.config(image=img)
    except:
        img_label.config(image='', text="Failed to load image")

def save_and_next(label_value):
    global current_index

    df.at[current_index, 'label'] = label_value
    current_index += 1

    if current_index >= len(df):
        messagebox.showinfo("End", "All data labeled. Click 'Save to CSV' to export.")
    else:
        load_data(current_index)

def save_to_csv():
    df.to_csv("labeled_output.csv", index=False)
    messagebox.showinfo("Saved", "Labels saved to labeled_output.csv")

# Buttons
btn_frame = tk.Frame(root)
btn_frame.pack(pady=10)

tk.Button(btn_frame, text="Mild", width=15, command=lambda: save_and_next("mild (no hurt)")).grid(row=0, column=0, padx=5)
tk.Button(btn_frame, text="Medium", width=15, command=lambda: save_and_next("medium (hurt)")).grid(row=0, column=1, padx=5)
tk.Button(btn_frame, text="Danger", width=15, command=lambda: save_and_next("danger (dead)")).grid(row=0, column=2, padx=5)
tk.Button(btn_frame, text="Save to CSV", width=48, command=save_to_csv).grid(row=1, column=0, columnspan=3, pady=10)

# Load first item
load_data(current_index)

root.mainloop()
